# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

path = ("../output_data/city_weather.csv")
df = pd.read_csv(path)
df.head()

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed
0,0,cidreira,55.44,-30.1811,-50.2056,93.0,10.0,8.03
1,1,arraial do cabo,67.93,-22.9661,-42.0278,76.0,74.0,16.58
2,2,pangnirtung,51.30,66.1451,-65.7125,75.0,99.0,3.87
3,3,zhigansk,61.12,66.7697,123.3711,54.0,56.0,11.77
4,4,taolanaro,0.00,0.0000,0.0000,0.0,0.0,0.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]
humidity = df["humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
print("Observations. The heatmap makes it easier to visualize places of high humidity, such as the east coast of Brazil, the west coast of Peru, and coastal California.")
print("Visualizing the map makes it seem likeit's much less likely for a city to have the highest humidity if it is not near the coast. For example, central Brazil and central United States.")
print("Just by looking at the chart,the small island of Mauritius (east of Africa) appears to have the highest humidity.")
fig

Observations. The heatmap makes it easier to visualize places of high humidity, such as the east coast of Brazil, the west coast of Peru, and coastal California.
Visualizing the map makes it seem likeit's much less likely for a city to have the highest humidity if it is not near the coast. For example, central Brazil and central United States.
Just by looking at the chart,the small island of Mauritius (east of Africa) appears to have the highest humidity.


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = pd.DataFrame(df[(df['temp(f)']<80) & (df['temp(f)']>70) & (df['humidity']<65) & (df['humidity']>55)])

hotel_df.shape

(13, 8)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ""
hotel_df['City'] = ""
hotel_df['Country'] = ""
hotel_df.head(20)

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,City,Country
28,28,moultonborough,73.63,43.7548,-71.3967,61.0,100.0,1.01,,,
114,114,bethel,72.82,41.3712,-73.4140,61.0,1.0,3.44,,,
170,170,thunder bay,79.56,48.4001,-89.3168,57.0,84.0,6.67,,,
347,347,alofi,76.89,-19.0595,-169.9187,61.0,75.0,8.05,,,
383,383,coacoyul,77.22,17.8058,-99.1100,57.0,76.0,2.13,,,
402,402,brentwood,74.34,40.7812,-73.2462,57.0,90.0,1.99,,,
435,435,half moon bay,73.00,37.4636,-122.4286,64.0,20.0,1.01,,,
465,465,pangai,73.33,-19.8000,-174.3500,58.0,26.0,10.83,,,
487,487,masuguru,72.73,-11.3667,38.4167,58.0,96.0,6.55,,,
491,491,touho,71.02,-20.7833,165.2333,64.0,29.0,8.86,,,


In [6]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df        

Missing field/result... skipping.


,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,City,Country
28,28,moultonborough,73.63,43.7548,-71.3967,61.0,100.0,1.01,Olde Orchard Farm,,
114,114,bethel,72.82,41.3712,-73.4140,61.0,1.0,3.44,Courtyard by Marriott Danbury,,
170,170,thunder bay,79.56,48.4001,-89.3168,57.0,84.0,6.67,Victoria Inn Hotel and Convention Centre,,
347,347,alofi,76.89,-19.0595,-169.9187,61.0,75.0,8.05,Taloa Heights,,
383,383,coacoyul,77.22,17.8058,-99.1100,57.0,76.0,2.13,Casa Abuelita Natalia,,
402,402,brentwood,74.34,40.7812,-73.2462,57.0,90.0,1.99,"Holiday Inn Express Hauppauge-Long Island, an ...",,
435,435,half moon bay,73.00,37.4636,-122.4286,64.0,20.0,1.01,"The Ritz-Carlton, Half Moon Bay",,
465,465,pangai,73.33,-19.8000,-174.3500,58.0,26.0,10.83,Ha'apai Beach Resort,,
487,487,masuguru,72.73,-11.3667,38.4167,58.0,96.0,6.55,,,
491,491,touho,71.02,-20.7833,165.2333,64.0,29.0,8.86,Auberge Timay Evasion,,


In [14]:
# params dictionary to update each iteration

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    params = {
    "latlng": f"{lat},{lng}", 
    "key": g_key
       }
    
    # change location each iteration while leaving original params in place
    params["laylng"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url_2 = "https://maps.googleapis.com/maps/api/geocode/json?"

    # make request and print url
    city_ctry = requests.get(base_url_2, params=params).json()
    #print(city_ctry)
    try:
        hotel_df.loc[index, "City"] = city_ctry["results"][0]["address_components"][2]["long_name"]
        hotel_df.loc[index, "Country"] = city_ctry["results"][0]["address_components"][7]["short_name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")    

        
hotel_df['City'].replace('', np.nan, inplace=True)
hotel_df['Country'].replace('', np.nan, inplace=True)
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['City'], inplace=True)
hotel_df.dropna(subset=['Country'], inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,City,Country
28,28,moultonborough,73.63,43.7548,-71.3967,61.0,100.0,1.01,Olde Orchard Farm,Moultonborough,3307
114,114,bethel,72.82,41.3712,-73.4140,61.0,1.0,3.44,Courtyard by Marriott Danbury,Bethel,US
170,170,thunder bay,79.56,48.4001,-89.3168,57.0,84.0,6.67,Victoria Inn Hotel and Convention Centre,Thunder Bay District,P7G 1P2
402,402,brentwood,74.34,40.7812,-73.2462,57.0,90.0,1.99,"Holiday Inn Express Hauppauge-Long Island, an ...",Brentwood,11717
435,435,half moon bay,73.00,37.4636,-122.4286,64.0,20.0,1.01,"The Ritz-Carlton, Half Moon Bay",Half Moon Bay,1918
515,515,shelburne,74.41,44.0787,-80.2041,62.0,88.0,6.02,The Retreat at the Farm,Horning's Mills,CA
603,603,grafton,73.54,42.2070,-71.6856,64.0,1.0,1.01,The Grafton Inn,Grafton,US


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [16]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))